In [1]:
# setup our environment

import pandas as pd
import numpy as np

DATA_FILE = "data/fake_or_real_news.csv"
TEST_SIZE = 0.2

In [2]:
# can't use GPU here -- network too big for syntactic features.
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
# define a function that allows us to evaluate our models

from sklearn.metrics import accuracy_score

def evaluate_model(predict_fun, X_train, y_train, X_test, y_test):
    '''
    evaluate the model, both training and testing errors are reported
    '''
    # training error
    y_predict_train = predict_fun(X_train)
    train_acc = accuracy_score(y_train,y_predict_train)
    
    # testing error
    y_predict_test = predict_fun(X_test)
    test_acc = accuracy_score(y_test,y_predict_test)
    
    return train_acc, test_acc

In [4]:
# estimate 95% confidence interval on error

# NOTE: based on conversation on stackexchange: 
# https://stats.stackexchange.com/questions/247551/how-to-determine-the-confidence-of-a-neural-network-prediction
# towards bottom of the page.

from math import sqrt

def error_conf(error, n):
    term = 1.96*sqrt((error*(1-error))/n)
    lb = error - term
    ub = error + term
    
    return lb, ub

In [5]:
# read in our data and preprocess it
# Note: the news items in the data set range from 0 words to 100,000 words
# we restrict ourselves to news items between 500 and 5,000 words.

df = pd.read_csv(DATA_FILE)
df.drop(labels=['id','title'], axis='columns', inplace=True)
# only select stories with between 500 and 5000 words
mask = list(df['text'].apply(lambda x: len(x) >= 500 and len(x) <= 5000))
df = df[mask]

In [6]:
print(df.shape)

(3510, 2)


In [7]:
# show that the sample data is pretty balanced
print(len(df[df['label'] == 'REAL']), len(df[df['label'] == 'FAKE']))

1472 2038


In [8]:
X = df['text']
y = df['label']

In [9]:
# convert labels to numeric labels
# NOTE: DNNs need numeric labels

def convert(x):
    if x == 'FAKE':
        return 0.0
    else:
        return 1.0


## Syntactic Features

In [10]:
# set up vector models for training and testing

from sklearn.feature_extraction.text import CountVectorizer

# data vectorizer
vectorizer = CountVectorizer(analyzer = "word", 
                             binary = True, 
                             min_df = 2,
                             stop_words='english')
docarray = vectorizer.fit_transform(X).toarray()
docterm = pd.DataFrame(docarray, columns=vectorizer.get_feature_names())

In [11]:
print(docterm.shape)

(3510, 23516)


In [12]:
# create training and test data
from sklearn.model_selection import train_test_split
docterm_train, docterm_test, y_train, y_test = train_test_split(docterm, y, test_size=TEST_SIZE)

### Naive Bayes

In [13]:
# Naive Bayes Model - train & test
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(docterm_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, docterm_train, y_train, docterm_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 96.62%
Testing Accuracy: 88.89%


In [15]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 86.56%-91.21%


### Random Forest

In [16]:
# Random Forest Model - train & test
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(min_samples_split=60)
model.fit(docterm_train, y_train)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=60,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [17]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, docterm_train, y_train, docterm_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 96.08%
Testing Accuracy: 83.76%


In [18]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 81.03%-86.49%


### Deep Neural Network

In [19]:
# DNN
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

# build the model
model = Sequential()
model.add(Dense(1024, input_dim=docterm_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(docterm_train, y_train.apply(convert),
          epochs=20,
          batch_size=128,
          validation_data=(docterm_test, y_test.apply(convert)))

Using TensorFlow backend.


Train on 2808 samples, validate on 702 samples
Epoch 1/20
2808/2808 [==============================] - 7s 3ms/step - loss: 0.6586 - acc: 0.6503 - val_loss: 0.4126 - val_acc: 0.8761
Epoch 2/20
2808/2808 [==============================] - 6s 2ms/step - loss: 0.2562 - acc: 0.9120 - val_loss: 0.1766 - val_acc: 0.9387
Epoch 3/20
2808/2808 [==============================] - 6s 2ms/step - loss: 0.0689 - acc: 0.9825 - val_loss: 0.2625 - val_acc: 0.9288
Epoch 4/20
2808/2808 [==============================] - 6s 2ms/step - loss: 0.0163 - acc: 0.9968 - val_loss: 0.4506 - val_acc: 0.9231
Epoch 5/20
2808/2808 [==============================] - 6s 2ms/step - loss: 0.0099 - acc: 0.9975 - val_loss: 0.4665 - val_acc: 0.9345
Epoch 6/20
2808/2808 [==============================] - 6s 2ms/step - loss: 0.0013 - acc: 0.9996 - val_loss: 0.5281 - val_acc: 0.9345
Epoch 7/20
2808/2808 [==============================] - 6s 2ms/step - loss: 2.1231e-04 - acc: 1.0000 - val_loss: 0.6426 - val_acc: 0.9316
Epoch 8/20


In [20]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict_classes, 
                                     docterm_train, 
                                     y_train.apply(convert), 
                                     docterm_test, 
                                     y_test.apply(convert))
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 100.00%
Testing Accuracy: 93.16%


In [21]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 91.30%-95.03%


## Semantic Features

In [22]:
# Load Spacy semantic model

import spacy

EMBEDDING_DIM = 300

# NOTE: for performance reasons disable everything in the pipeline except the tokenizer
nlp = spacy.load('en_core_web_lg', disable=['parser', 'tagger', 'ner', 'textcat'])

In [23]:
# preprocess text for semantic features

def embed(X):
    '''
    x is a list of strings and embed will compute
    an embedding vector for each and return an array
    of shape (len(x),EMBEDDING_DIM)
    '''
    vectors = []
    text_array = np.array(X)

    print(text_array.shape)
    
    for i in range(text_array.shape[0]):
        vector = nlp(text_array[i]).vector
        vectors.append(vector)
    
    return pd.DataFrame(vectors)

vectors = embed(X)

(3510,)


In [24]:
# create training and test data
from sklearn.model_selection import train_test_split
vectors_train, vectors_test, y_train, y_test = train_test_split(vectors, y, test_size=TEST_SIZE)

### Naive Bayes

In [25]:
# Naive Bayes Model 
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(vectors_train,y_train)

GaussianNB(priors=None)

In [26]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, vectors_train, y_train, vectors_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 73.65%
Testing Accuracy: 74.36%


In [27]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 71.13%-77.59%


### Random Forest

In [28]:
# Random Forest Model 
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(min_samples_split=10)
model.fit(vectors_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, vectors_train, y_train, vectors_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 98.25%
Testing Accuracy: 82.34%


In [30]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 79.52%-85.16%


### Deep Neural Network

In [31]:
# DNN - MLP
from keras.models import Sequential
from keras.layers import Dense, Dropout

# build the model
model = Sequential()
model.add(Dense(256, input_dim=EMBEDDING_DIM, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [32]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(vectors_train, y_train.apply(convert),
          epochs=15,
          batch_size=128,
          validation_data=(vectors_test, y_test.apply(convert)))

Train on 2808 samples, validate on 702 samples
Epoch 1/15
2808/2808 [==============================] - 1s 190us/step - loss: 0.6838 - acc: 0.5577 - val_loss: 0.6574 - val_acc: 0.5798
Epoch 2/15
2808/2808 [==============================] - 0s 43us/step - loss: 0.6375 - acc: 0.6140 - val_loss: 0.5383 - val_acc: 0.7892
Epoch 3/15
2808/2808 [==============================] - 0s 42us/step - loss: 0.5494 - acc: 0.7301 - val_loss: 0.4254 - val_acc: 0.8376
Epoch 4/15
2808/2808 [==============================] - 0s 42us/step - loss: 0.4773 - acc: 0.7910 - val_loss: 0.3682 - val_acc: 0.8533
Epoch 5/15
2808/2808 [==============================] - 0s 45us/step - loss: 0.4330 - acc: 0.8102 - val_loss: 0.3752 - val_acc: 0.8433
Epoch 6/15
2808/2808 [==============================] - 0s 42us/step - loss: 0.4076 - acc: 0.8237 - val_loss: 0.3080 - val_acc: 0.8746
Epoch 7/15
2808/2808 [==============================] - 0s 42us/step - loss: 0.3749 - acc: 0.8383 - val_loss: 0.3017 - val_acc: 0.8732
Epoch 8

In [33]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict_classes, 
                                     vectors_train, 
                                     y_train.apply(convert), 
                                     vectors_test, 
                                     y_test.apply(convert))
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 89.74%
Testing Accuracy: 89.89%


In [34]:
# estimating 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 87.66%-92.12%
